In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

## Read in data

In [3]:
# read data from covid_lies.csv dataset into dataframe
df = pd.read_csv('./data/covid_lies.csv')

# seperate out text data and labels 
input_text = df['misconception'].to_numpy()
input_label = df['label'].to_numpy()

print(f'Sample Text Input: {input_text[0]}')
print(f'Sample Text Label: {input_label[0]}')

Sample Text Input: Coronavirus is genetically engineered.
Sample Text Label: na


## Preprocess input data

In [ ]:
# handle setting all values to our preferred casing style
# removing erroneous punctuation
# handle stop words
# think about standardizing number formats, not fully necessary (careful with 'covid-19')

## Tokenize input text data

## Form embeddings for input data

## Define model using PyTorch

## Save Model Weights

## Evaluate Model